In [ ]:
!python -m pip install -r ../requirements.txt

In [ ]:
import PyPDF2
import pdf2image
from PIL import Image
import pytesseract
from transformers import pipeline
import docx

In [ ]:
import os

if "OS" in os.environ.keys() and "WINDOWS" in os.environ["OS"].upper():
    OS="WINDOWS"
    pytesseract.pytesseract.tesseract_cmd = (
        r"C:\Users\conrad.thiounn\AppData\Local\Programs\Tesseract-OCR\tesseract.exe"
    )
    POPPLER_PATH = r"C:\Users\conrad.thiounn\Desktop\poppler-23.01.0\Library\bin"
else:
    OS="UNIX"
    pytesseract.pytesseract.tesseract_cmd = (
        r"/usr/bin/tesseract"
    )
    POPPLER_PATH = r"/usr/share/poppler"

In [ ]:
DATA_BASE_DIR="./"
NAME_OF_PDF_FILE_TO_READ="arrete_jury_composition_Administrateur_ENS_2023.pdf"
FULL_PATH_PDF_FILE=DATA_BASE_DIR+NAME_OF_PDF_FILE_TO_READ

In [ ]:
# https://stackoverflow.com/questions/34837707/how-to-extract-text-from-a-pdf-file
def read_pdf(pdf_file_path: str) -> str:
    # First, create an opener which accepts a PDF file path
    opener = open(pdf_file_path, "rb")
    # Second, read the opened file
    pdf_file_reader = PyPDF2.PdfReader(opener)
    number_of_pages = len(pdf_file_reader.pages)
    text = ""
    for i in range(number_of_pages):
        page = pdf_file_reader.pages[i]
        text += page.extract_text()
    return text

In [ ]:
# https://stackoverflow.com/questions/64048828/pytesseract-gives-an-error-permissionerror-winerror-5-access-is-denied
def read_and_print_pdf_with_ocr(pdf_file_path: str,lang:str=None) -> None:
    def pdf_to_img(pdf_file):
        return pdf2image.convert_from_path(pdf_file)

    def ocr_core(file):
        if lang:
            text = pytesseract.image_to_string(file, lang=lang)
        else:
            text = pytesseract.image_to_string(file)
        return text

    def print_pages(pdf_file):
        text=""
        images = pdf_to_img(pdf_file)
        for pg, img in enumerate(images):
            text+=ocr_core(img)
        return text

    return print_pages(pdf_file_path)

In [ ]:
# https://stackoverflow.com/questions/58613880/python-grab-all-text-in-docx-and-dump-into-txt
def read_txt_from_docx(filename: str) -> str:
    doc = docx.Document(filename)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)
    return "\n".join(fullText)

In [ ]:
txt_from_ocr=read_and_print_pdf_with_ocr(FULL_PATH_PDF_FILE)

In [ ]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

ARTICLE = """
Rapidité, qualité et lisibilité, tels sont les besoins des citoyens en matière de prise d'informations. L'Insee et les Services Statistiques Ministériels (SSM), dont la Dares, ont pour ambition de réduire les délais de production et de diffusion des statistiques, comme déclaré dans le plan stratégique "Insee 2025". Pour cela, les conclusions de ce plan mettent en exergue la nécessité de mobiliser de nouvelles sources, ainsi que d'enrichir les sources existantes pour permettre de réduire les délais et de diminuer les coûts.
La direction de l'animation de la recherche, des études et des statistiques (Dares), SSM du ministère en charge du travail et de l'emploi, produit et diffuse à un rythme régulier des chiffres en matière d'emploi, de chômage, de salaire et de conditions de travail, tout en étant accompagnés d'études pour comprendre le chiffre. Pour comprendre le marché du travail, il est important de comprendre chaque pan du tissu économique, chaque secteur et branche de l'économie française. L'observation des accords entre agents économiques permet de mieux saisir les enjeux d'aujourd'hui et de demain.
En grand nombre, une matière première évidente est la juridiction du monde de l'entreprise, composée d'accords collectifs, qu'ils soient de branche ou d'entreprise. Ces accords complètent le code du travail et sont le fruit d'une négociation mûrement réfléchie entre acteurs du monde professionnel. Aujourd'hui, ces accords sont largement sous-exploitées dans la production et l'analyse des chiffres produits par la Dares, compte tenu de la complexité à traiter de manière automatique des documents numériques hétérogènes et à y piocher l'information pertinente.
Notamment, reconstituer un sommaire depuis un document numérique, scanné ou typographié, permettrait de gagner en lisibilité et de donner accès rapidement à une vue d'ensemble. Pour aller plus loin, produire un résumé permettrait de faciliter l'analyse de données et donc la production du chiffre.
Les dernières méthodes d'apprentissage statistique en traitement automatique du langage permettent de réduire cette complexité, en proposant des modèles génériques déjà entraînés. Il suffirait de réutiliser ces modèles en composant intelligemment, de les améliorer avec des données spécifiques.
Concevoir et produire un modèle en adéquation avec le besoin exprimé par la Dares est une première étape cruciale. Néanmoins, pour pérenniser l'investissement, la seconde étape est de le mettre en production dans les règles de l'art.
"""
print(summarizer(ARTICLE, max_length=1000, min_length=300, do_sample=True))

In [ ]:
import spacy

!python -m spacy download fr_core_news_lg

In [ ]:
nlp = spacy.load("fr_core_news_lg")
sentence = txt_from_ocr
doc = nlp(sentence)

for ent in doc.ents:
    print(ent.text," - ", ent.label_)